In [1]:
# !pip install sentencepiece
# !pip  install transformers
# !pip install pytorch-ignite
# !pip install datasets



In [1]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import random
import numpy as np
import torch.nn as nn
import torch
from transformers import AutoModel
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score, classification_report
from transformers import AdamW, get_linear_schedule_with_warmup
# Any results you write to the current directory are saved as output.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW
from tqdm import tqdm
from argparse import ArgumentParser
# from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
# from ignite.metrics import Accuracy, Loss
# from ignite.engine.engine import Engine, State, Events
# from ignite.handlers import EarlyStopping
# from ignite.contrib.handlers import TensorboardLogger, ProgressBar
# from ignite.utils import convert_tensor
from torch.optim.lr_scheduler import ExponentialLR
import warnings  
warnings.filterwarnings('ignore')
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
from transformers import AutoTokenizer, AutoModel, AdamW
import random
import os
from urllib import request
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from transformers import AutoModel, PreTrainedModel
from transformers import BertModel, BertPreTrainedModel, BertForMaskedLM
from transformers.models.bert.modeling_bert import BertOnlyMLMHead, BertLayer

# Preprocessing

In [2]:
class TrainDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['arabic_original_text_deomji'].values
        self.label=df['label'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        # summary = self.summary[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
                            
        target = self.label[index]
        
        text_ids = inputs_text['input_ids']
        text_mask = inputs_text['attention_mask']
#         token_type_ids=inputs_text['token_type_ids']
       
        
        
        return {
            
            'text_ids': torch.tensor(text_ids, dtype=torch.long),
            'text_mask': torch.tensor(text_mask, dtype=torch.long),
#              'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.float)
        }

# Losses

In [3]:
class F1_Loss(nn.Module):
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. epsilon <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    '''
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        # assert y_pred.ndim == 2
        # assert y_true.ndim == 1
        # print(y_pred.shape)
        # print(y_true.shape)
        # y_pred[y_pred<0.5]=0
        # y_pred[y_pred>=0.5]=0


        
        y_true_one_hot = F.one_hot(y_true.to(torch.int64), 3).to(torch.float32)
        # y_pred_one_hot = F.one_hot(y_pred.to(torch.int64), 2).to(torch.float32)
        
        tp = (y_true_one_hot * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true_one_hot) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true_one_hot) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true_one_hot * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        f1=f1.detach()
        # print(f1.shape)
        # y_pred=y_pred.reshape((y_pred.shape[0], 1))
        # y_true=y_true.reshape((y_true.shape[0], 1))

        # p1=y_true*(math.log(sigmoid(y_pred)))*(1-f1)[1]
        # p0=(1-y_true)*math.log(1-sigmoid(y_pred))*(1-f1)[0]


        # y_true_one_hot = F.one_hot(y_true.to(torch.int64), 2)
        # print(y_pred)
        # print(y_true_one_hot)
        CE =torch.nn.CrossEntropyLoss(weight=( 1 - f1))(y_pred, y_true)
        # loss = ( 1 - f1)  * CE
        return  CE.mean()

In [4]:
class Recall_Loss(nn.Module):
    '''Calculate Recall score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. epsilon <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    '''
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        # assert y_pred.ndim == 2
        # assert y_true.ndim == 1
        # print(y_pred.shape)
        # print(y_true.shape)
        # y_pred[y_pred<0.5]=0
        # y_pred[y_pred>=0.5]=0


        
        y_true_one_hot = F.one_hot(y_true.to(torch.int64), 18).to(torch.float32)
        # y_pred_one_hot = F.one_hot(y_pred.to(torch.int64), 2).to(torch.float32)
        
        tp = (y_true_one_hot * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true_one_hot) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true_one_hot) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true_one_hot * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        # f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        # f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        # f1=f1.detach()
        # print(f1.shape)
        # y_pred=y_pred.reshape((y_pred.shape[0], 1))
        # y_true=y_true.reshape((y_true.shape[0], 1))

        # p1=y_true*(math.log(sigmoid(y_pred)))*(1-f1)[1]
        # p0=(1-y_true)*math.log(1-sigmoid(y_pred))*(1-f1)[0]


        # y_true_one_hot = F.one_hot(y_true.to(torch.int64), 2)
        # print(y_pred)
        # print(y_true_one_hot)
        recall=recall.detach()
        CE =torch.nn.CrossEntropyLoss(weight=( 1 - recall))(y_pred, y_true_one_hot)
        # loss = ( 1 - f1)  * CE
        return  CE.mean()

In [5]:
## Based on https://github.com/AbdelkaderMH/iSarcasmEval/blob/8f28f24ebfb641415a604329ed859506ae687148/focal_loss.py
class BinaryFocalLoss(nn.Module):
    """
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
        Focal_Loss= -1*alpha*(1-pt)*log(pt)
    :param alpha: (tensor) 3D or 4D the scalar factor for this criterion
    :param gamma: (float,double) gamma > 0 reduces the relative loss for well-classified examples (p>0.5) putting more
                    focus on hard misclassified example
    :param reduction: `none`|`mean`|`sum`
    :param **kwargs
        balance_index: (int) balance class index, should be specific when alpha is float
    """

    def __init__(self, alpha=3, gamma=2, ignore_index=None, reduction='mean', **kwargs):
        super(BinaryFocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smooth = 1e-6  # set '1e-4' when train with FP16
        self.ignore_index = ignore_index
        self.reduction = reduction

        assert self.reduction in ['none', 'mean', 'sum']

        # if self.alpha is None:
        #     self.alpha = torch.ones(2)
        # elif isinstance(self.alpha, (list, np.ndarray)):
        #     self.alpha = np.asarray(self.alpha)
        #     self.alpha = np.reshape(self.alpha, (2))
        #     assert self.alpha.shape[0] == 2, \
        #         'the `alpha` shape is not match the number of class'
        # elif isinstance(self.alpha, (float, int)):
        #     self.alpha = np.asarray([self.alpha, 1.0 - self.alpha], dtype=np.float).view(2)

        # else:
        #     raise TypeError('{} not supported'.format(type(self.alpha)))

    def forward(self, output, target):
        prob = torch.sigmoid(output)
        prob = torch.clamp(prob, self.smooth, 1.0 - self.smooth)

        valid_mask = None
        if self.ignore_index is not None:
            valid_mask = (target != self.ignore_index).float()

        pos_mask = (target == 1).float()
        neg_mask = (target == 0).float()
        if valid_mask is not None:
            pos_mask = pos_mask * valid_mask
            neg_mask = neg_mask * valid_mask

        pos_weight = (pos_mask * torch.pow(1 - prob, self.gamma)).detach()
        pos_loss = -pos_weight * torch.log(prob)  # / (torch.sum(pos_weight) + 1e-4)

        neg_weight = (neg_mask * torch.pow(prob, self.gamma)).detach()
        neg_loss = -self.alpha * neg_weight * F.logsigmoid(-output)  # / (torch.sum(neg_weight) + 1e-4)
        loss = pos_loss + neg_loss
        loss = loss.mean()
        return loss


class FocalLoss_Ori(nn.Module):
    """
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
    Focal_Loss= -1*alpha*((1-pt)**gamma)*log(pt)
    Args:
        num_class: number of classes
        alpha: class balance factor
        gamma:
        ignore_index:
        reduction:
    """

    def __init__(self, num_class, alpha=None, gamma=2, ignore_index=None, reduction='mean'):
        super(FocalLoss_Ori, self).__init__()
        self.num_class = num_class
        self.gamma = gamma
        self.reduction = reduction
        self.smooth = 1e-4
        self.ignore_index = ignore_index
        self.alpha = alpha
        if alpha is None:
            self.alpha = torch.ones(num_class, )
        elif isinstance(alpha, (int, float)):
            self.alpha = torch.as_tensor([alpha] * num_class)
        elif isinstance(alpha, (list, np.ndarray)):
            self.alpha = torch.as_tensor(alpha)
        if self.alpha.shape[0] != num_class:
            raise RuntimeError('the length not equal to number of class')

        # if isinstance(self.alpha, (list, tuple, np.ndarray)):
        #     assert len(self.alpha) == self.num_class
        #     self.alpha = torch.Tensor(list(self.alpha))
        # elif isinstance(self.alpha, (float, int)):
        #     assert 0 < self.alpha < 1.0, 'alpha should be in `(0,1)`)'
        #     assert balance_index > -1
        #     alpha = torch.ones((self.num_class))
        #     alpha *= 1 - self.alpha
        #     alpha[balance_index] = self.alpha
        #     self.alpha = alpha
        # elif isinstance(self.alpha, torch.Tensor):
        #     self.alpha = self.alpha
        # else:
        #     raise TypeError('Not support alpha type, expect `int|float|list|tuple|torch.Tensor`')

    def forward(self, logit, target):
        # assert isinstance(self.alpha,torch.Tensor)\
        N, C = logit.shape[:2]
        alpha = self.alpha.to(logit.device)
        prob = F.softmax(logit, dim=1)
        if prob.dim() > 2:
            # N,C,d1,d2 -> N,C,m (m=d1*d2*...)
            prob = prob.view(N, C, -1)
            prob = prob.transpose(1, 2).contiguous()  # [N,C,d1*d2..] -> [N,d1*d2..,C]
            prob = prob.view(-1, prob.size(-1))  # [N,d1*d2..,C]-> [N*d1*d2..,C]
        ori_shp = target.shape
        target = target.view(-1, 1)  # [N,d1,d2,...]->[N*d1*d2*...,1]
        valid_mask = None
        if self.ignore_index is not None:
            valid_mask = target != self.ignore_index
            target = target * valid_mask

        # ----------memory saving way--------
        prob = prob.gather(1, target).view(-1) + self.smooth  # avoid nan
        logpt = torch.log(prob)
        # alpha_class = alpha.gather(0, target.view(-1))
        alpha_class = alpha[target.squeeze().long()]
        class_weight = -alpha_class * torch.pow(torch.sub(1.0, prob), self.gamma)
        loss = class_weight * logpt
        if valid_mask is not None:
            loss = loss * valid_mask.squeeze()

        if self.reduction == 'mean':
            loss = loss.mean()
            if valid_mask is not None:
                loss = loss.sum() / valid_mask.sum()
        elif self.reduction == 'none':
            loss = loss.view(ori_shp)
        return loss



def weighted_binary_cross_entropy(input, targets, pos_weight, weight=None, size_average=True, reduce=True):
    """
    Args:
        sigmoid_x: predicted probability of size [N,C], N sample and C Class. Eg. Must be in range of [0,1], i.e. Output from Sigmoid.
        targets: true value, one-hot-like vector of size [N,C]
        pos_weight: Weight for postive sample
    """
    sigmoid_x = torch.sigmoid(input)
    if not (targets.size() == sigmoid_x.size()):
        raise ValueError("Target size ({}) must be the same as input size ({})".format(targets.size(), sigmoid_x.size()))

    loss = -pos_weight* targets * sigmoid_x.log() - (1-targets)*(1-sigmoid_x).log()

    if weight is not None:
        loss = loss * weight

    if not reduce:
        return loss
    elif size_average:
        return loss.mean()
    else:
        return loss.sum()

class WeightedBCELoss(nn.Module):
    def __init__(self, pos_weight= 1, weight=None, PosWeightIsDynamic= True, WeightIsDynamic= False, size_average=True, reduce=True):
        """
        Args:
            pos_weight = Weight for postive samples. Size [1,C]
            weight = Weight for Each class. Size [1,C]
            PosWeightIsDynamic: If True, the pos_weight is computed on each batch. If pos_weight is None, then it remains None.
            WeightIsDynamic: If True, the weight is computed on each batch. If weight is None, then it remains None.
        """
        super().__init__()

        #self.register_buffer('weight', weight)
        #self.register_buffer('pos_weight', pos_weight)
        self.size_average = size_average
        self.reduce = reduce
        self.PosWeightIsDynamic = PosWeightIsDynamic

    def forward(self, input, target):
        # pos_weight = Variable(self.pos_weight) if not isinstance(self.pos_weight, Variable) else self.pos_weight
        if self.PosWeightIsDynamic:
            positive_counts = target.sum(dim=0)
            nBatch = len(target)
            self.pos_weight = (nBatch - positive_counts)/(positive_counts +1e-5)


        return weighted_binary_cross_entropy(input, target,
                                                self.pos_weight,
                                                weight=None,
                                                size_average=self.size_average,
                                                reduce=self.reduce)


class WeightedCELoss(nn.Module):
    def __init__(self, size_average=True, reduce=True):
        """
        Args:
            pos_weight = Weight for postive samples. Size [1,C]
            weight = Weight for Each class. Size [1,C]
            PosWeightIsDynamic: If True, the pos_weight is computed on each batch. If pos_weight is None, then it remains None.
            WeightIsDynamic: If True, the weight is computed on each batch. If weight is None, then it remains None.
        """
        super().__init__()

        self.size_average = size_average
        self.reduce = reduce

    def forward(self, input, target):
        positive_counts = target.sum(dim=0)
        nBatch = len(target)
        pos_weight = (nBatch - positive_counts)/(positive_counts +1e-5)
        neg_count = nBatch - positive_counts
        neg_weight = (nBatch - neg_count)/(neg_count +1e-5)

        weight = torch.tensor([neg_weight, pos_weight], device=target.device)
  


        return F.cross_entropy(input, target, weight=weight)



class FLMultiLoss(nn.Module):
    def __init__(self, gamma= 2):
        """
        Args:
            pos_weight = Weight for postive samples. Size [1,C]
            weight = Weight for Each class. Size [1,C]
            PosWeightIsDynamic: If True, the pos_weight is computed on each batch. If pos_weight is None, then it remains None.
            WeightIsDynamic: If True, the weight is computed on each batch. If weight is None, then it remains None.
        """
        super().__init__()

        self.gamma = gamma
    def forward(self, input, target):


        return focal_binary_cross_entropy(input, target, gamma=2)

def EntropyLoss(input_):
    mask = input_.ge(0.000001)
    mask_out = torch.masked_select(input_, mask)
    entropy = -(torch.sum(mask_out * torch.log(mask_out)))
    return entropy / float(input_.size(0))

def focal_binary_cross_entropy(logits, targets, gamma=2):
    num_label = targets.shape[1]
    l = logits.reshape(-1)
    t = targets.reshape(-1)
    p = torch.sigmoid(l)
    p = torch.where(t >= 0.5, p, 1-p)
    logp = - torch.log(torch.clamp(p, 1e-4, 1-1e-4))
    loss = logp*((1-p)**gamma)
    loss = num_label*loss.mean()
    return loss

In [6]:
from typing import Optional, Sequence

import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F


class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.
    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.
    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.
        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cuda',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.
    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.
    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl

# Models

In [7]:
def freeze(module):
    """
    Freezes module's parameters.
    """
    for parameter in module.parameters():
        parameter.requires_grad = False

def get_freezed_parameters(module):
    """
    Returns names of freezed parameters of the given module.
    """
    freezed_parameters = []
    for name, parameter in module.named_parameters():
        if not parameter.requires_grad:
            freezed_parameters.append(name)

In [8]:
import torch.nn as nn
import torch
from transformers import AutoModel
import torch.nn.functional as F

def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1 or classname.find('ConvTranspose2d') != -1:
        nn.init.kaiming_uniform_(m.weight)
        nn.init.zeros_(m.bias)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.zeros_(m.bias)
    elif classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


class AttentionWithContext(nn.Module):
    def __init__(self, hidden_dim):
        super(AttentionWithContext, self).__init__()

        self.attn = nn.Linear(hidden_dim, hidden_dim)
        self.contx = nn.Linear(hidden_dim, 1, bias=False)
        #self.apply(init_weights)
    def forward(self, inp):
        u = torch.tanh_(self.attn(inp))
        a = F.softmax(self.contx(u))
        s = (a * inp).sum(1)
        return s


class TransformerLayer(nn.Module):
    def __init__(self,
                 pretrained_path='aubmindlab/bert-base-arabert'):
        super(TransformerLayer, self).__init__()

        
        self.transformer = AutoModel.from_pretrained(pretrained_path, output_hidden_states=True)


    def forward(self, input_ids=None, attention_mask=None):
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask
            , output_hidden_states=True
        )
        #(output_last_layer, pooled_cls, (output_layers))
        #output[0] (8, seqlen=64, 768) cls [8, 768] ( 12 (8, seqlen=64, 768))

        return outputs

    def output_num(self):
        return self.transformer.config.hidden_size

class ATTClassifier(nn.Module):
    def __init__(self, in_feature, class_num=1, dropout_prob=0.2):
        super(ATTClassifier, self).__init__()
        self.attention = AttentionWithContext(in_feature)

        self.Classifier = nn.Sequential(
            nn.Linear(2 * in_feature, 512),
            nn.Dropout(dropout_prob),
            nn.ReLU(),
            nn.Linear(512, class_num)
        )

        self.apply(init_weights)

    def forward(self, x):
        att = self.attention(x[0]) #(X[0] (bs, seqlenght, embedD) att = \sum_i alpha_i x[0][i]

        xx = torch.cat([att, x[1]], 1)

        out = self.Classifier(xx)
        return out
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.kaiming_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim 
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / (torch.sum(a, 1, keepdim=True) + 1e-10)

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights=None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
            torch.tensor([1] * (num_hidden_layers + 1 - layer_start), dtype=torch.float)
        )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start:, :, :, :]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor * all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average
class NADIModel(nn.Module):
  def __init__(self, pretrained_path='aubmindlab/bert-base-arabert',in_feature=768, class_num=3, dropout_prob=0.2):
        super(NADIModel, self).__init__()
        self.bert=AutoModel.from_pretrained(pretrained_path, output_hidden_states=True).to('cuda')
        freeze(self.bert.embeddings)
#         freeze(self.bert.encoder.layer[:4])
        self.config = AutoConfig.from_pretrained(pretrained_path)
        self.weighted_pooler = WeightedLayerPooling(num_hidden_layers=self.config.num_hidden_layers, layer_start=4)
        self.att=Attention(self.config.hidden_size, 128).to('cuda')
        self.classifier=nn.Linear(self.config.hidden_size, class_num)
  def forward(self, ids,mask):
    output=self.bert(ids,mask)
    all_hidden_states = torch.stack(output.hidden_states)
    weighted_pooling_embeddings = (self.weighted_pooler(all_hidden_states)) # For WeightedLayerPooling
    # weighted_pooling_embeddings = weighted_pooling_embeddings[:, 0]
    # print(weighted_pooling_embeddings.shape)
    output=self.att(weighted_pooling_embeddings)
    output=self.classifier(output)
    # output=torch.softmax(output, dim=1)
    return output
 

# Train, Validation

In [9]:
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """
    Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
    """

    def __init__(self, dataset, indices=None, num_samples=None):
        # if indices is not provided,
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset['label']))) \
            if indices is None else indices

        # if num_samples is not provided,
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples

        # distribution of classes in the dataset
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1

        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)] for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, id_):
        return dataset['label'][id_]

    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

In [10]:
def criterion(outputs1,  targets):

    criterion = nn.CrossEntropyLoss()
#     criteria_2=FocalLoss()
    loss = criterion(outputs1, targets)
    return loss

In [11]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        
        text_ids = data['text_ids'].to(device, dtype = torch.long)
        text_mask = data['text_mask'].to(device, dtype = torch.long)
#         token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.long)
        
        batch_size = text_ids.size(0)
        # print(targets)

        outputs = model(text_ids, text_mask)
        # print(outputs.shape)
        
        # print(outputs.shape)

        
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [12]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    prediction=[]
    true_prediction=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        
        text_ids = data['text_ids'].to(device, dtype = torch.long)
        text_mask = data['text_mask'].to(device, dtype = torch.long)
#         token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#         mask_pos =data['mask_pos'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.long)
        
        batch_size = text_ids.size(0)

        outputs = model(text_ids, text_mask)
        prediction.append(F.softmax(outputs).to('cpu').numpy())
        true_prediction.append(targets.to('cpu').numpy())

        # outputs = outputs.argmax(dim=1)
        
        loss = criterion(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr']) 
    
    gc.collect()
#     print(np.array(prediction)[0][0])
    prediction = np.concatenate(prediction)
    true_prediction = np.concatenate(true_prediction)
    prediction=np.argmax(np.array(prediction),axis=1)
    print(print_statistics(np.array(true_prediction),prediction))
    print(f1_score(np.array(true_prediction),prediction, average='macro'))

    
    return epoch_loss, f1_score(np.array(true_prediction),prediction, average='macro')

In [13]:
from sklearn.metrics import jaccard_score,f1_score,accuracy_score,recall_score,precision_score,classification_report
def print_statistics(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    precision =precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')
    f_score = f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(classification_report(y, y_pred))
    return accuracy, precision, recall, f_score

In [14]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold):
    # To automatically log gradients
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = 0
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss,f1_score = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
       
        
        # deep copy the model
        if f1_score >= best_epoch_loss:
            print(f"Validation Loss Improved ({best_epoch_loss} ---> {f1_score})")
            best_epoch_loss = f1_score
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Marbert/Deberta_large-{fold}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print("Model Saved")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [15]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [16]:
def prepare_loaders(train,valid):
#     df_train = train[train.kfold != fold].reset_index(drop=True)
#     df_valid = train[train.kfold == fold].reset_index(drop=True)
    sampler = ImbalancedDatasetSampler(train)
    
    train_dataset = TrainDataset(train, tokenizer=tokenizer, max_length=CONFIG['max_length'])
    valid_dataset = TrainDataset(valid, tokenizer=tokenizer, max_length=CONFIG['max_length'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, sampler=sampler, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

# Train

In [17]:
!pwd

/home/ahmed/Downloads/Imagenet_VLCQ/Nadi


In [18]:
# from google.colab import drive
# drive.mount('/content/drive')

In [19]:
train = pd.read_csv('/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Dataset/train_subtask2_with_features_modified.csv')
valid = pd.read_csv('/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Dataset/valid_subtask2_with_features_modified.csv')

In [20]:
train.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,Neutral Sentiment,sentiment,...,count_nouns,count_pronun,count_adjct,entities,count_profane,translated text,english_translated text,english_demoji_text,arabic_original_text_deomji,arabic_translated_text_deomji
0,أنا وطن بحاجة إلى شعب . . تعالي واسكنيني .,2,4.0,1.0,3.0,0.0,0.371753,0.215986,1.000,0.155767,...,0,3,0,NaN,0,أنا وطن يحتاج إلى شعب. . تعال واجعلني أعيش.,I am a homeland that needs a people. . Come an...,I am a homeland that needs a people. . Come an...,أنا وطن بحاجة إلى شعب . . تعالي واسكنيني .,أنا وطن يحتاج إلى شعب. . تعال واجعلني أعيش.
1,#احذرو_الرافضه_غدر_وخيانه الرافضة شر وبال على ...,0,14.0,3.0,3.0,1.0,0.981253,0.479370,0.835,0.501883,...,6,3,1,Shiites Islamic Abu Loula Al -Majusi,0,#beware من الرفضين وتيتيير وتدريزيه ، والشيعة ...,#Beware of the rejectionists_their and his tre...,#Beware of the rejectionists_their and his tre...,#احذرو_الرافضه_غدر_وخيانه الرافضة شر وبال على ...,#beware من الرفضين وتيتيير وتدريزيه ، والشيعة ...
2,كلما راجعت أحلام الصبا قلت يا ليت الصبا لم يزل...,2,10.0,5.0,8.0,3.0,0.673062,1.822552,0.690,-1.149490,...,3,6,1,NaN,0,كلما راجعت أحلام الطفولة ، قلت ، أتمنى أن يكون...,"Whenever I reviewed the dreams of boyhood, I s...","Whenever I reviewed the dreams of boyhood, I s...",كلما راجعت أحلام الصبا قلت يا ليت الصبا لم يزل...,كلما راجعت أحلام الطفولة ، قلت ، أتمنى أن يكون...
3,مؤلم جدا عندما يبتعد أقرب أصدقائك بسبب أصدقاء جدد,0,4.0,2.0,1.0,1.0,0.333239,0.718892,0.460,-0.385653,...,0,1,3,NaN,0,مؤلم للغاية عندما ينتقل أصدقاؤك المقربين من ال...,Very painful when your closest friends move aw...,Very painful when your closest friends move aw...,مؤلم جدا عندما يبتعد أقرب أصدقائك بسبب أصدقاء جدد,مؤلم للغاية عندما ينتقل أصدقاؤك المقربين من ال...
4,USER معنه ليلحين احس مابردت چبدي عدل بس اهم شي...,0,8.0,4.0,5.0,1.0,0.927923,0.402966,0.667,0.524956,...,4,4,1,Dear Florezy 💛,0,المستخدم ، مع سبب يجعلني أشعر بما أريد أن أجعل...,"User, with a reason to make me feel what I wan...","User, with a reason to make me feel what I wan...",USER معنه ليلحين احس مابردت چبدي عدل بس اهم شي...,المستخدم ، مع سبب يجعلني أشعر بما أريد أن أجعل...


In [21]:
len(train)

1500

In [22]:
# train['#3_label'].nunique()

In [23]:
# valid['#3_label'].nunique()

In [24]:
# df_train=pd.concat([train,valid])

In [25]:
# num_classes=train['#3_label'].nunique()

In [25]:
CONFIG = {"seed": 42,
          "epochs": 100,
          "train_batch_size": 4,
          "valid_batch_size": 8,
          "max_length": 128,
          "learning_rate": 4e-7,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-8,
          "T_max": 500,
          "weight_decay": 1e-8,
          "n_fold": 5,
          "n_accumulate": 1,
          "num_classes": 18,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          }

In [26]:
tokenizer= AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")


In [27]:
train.reset_index(inplace=True)

In [28]:
# skf = StratifiedKFold(n_splits=CONFIG['n_fold'], shuffle=True, random_state=CONFIG['seed'])

# for fold, ( _, val_) in enumerate(skf.split(X=train, y=train['#3_label'])):
#     train.loc[val_ , "kfold"] = int(fold)
    
# train["kfold"] = train["kfold"].astype(int)

In [29]:
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# train['#3_label'] = le.fit_transform(train['#3_label'].values)

In [30]:
# valid['#3_label'] = le.transform(valid['#3_label'].values)

In [31]:
# for fold in range(0, CONFIG['n_fold']):
#     print(f"====== Fold: {fold} ======")

    
# Create Dataloaders
train_loader, valid_loader = prepare_loaders(train,valid)

model = NADIModel( "CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")
model.to(CONFIG['device'])
torch.cuda.empty_cache()

# Define Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

model, history = run_training(model, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=CONFIG['epochs'],
                              fold=0)


del model, history, train_loader, valid_loader
_ = gc.collect()
print()

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[INFO] Using GPU: NVIDIA GeForce RTX 3080 Ti



100%|███████████████████████████████████████████| 375/375 [00:12<00:00, 30.23it/s, Epoch=1, LR=6.71e-8, Train_Loss=1.13]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|█████████████████████████████████████████████| 63/63 [00:00<00:00, 83.26it/s, Epoch=1, LR=6.71e-8, Valid_Loss=1.07]


Accuracy: 0.400
Precision: 0.401
Recall: 0.400
F_score: 0.312

              precision    recall  f1-score   support

           0       0.39      0.87      0.54       190
           1       0.29      0.04      0.08       113
           2       0.47      0.15      0.22       197

    accuracy                           0.40       500
   macro avg       0.39      0.36      0.28       500
weighted avg       0.40      0.40      0.31       500

(0.4, 0.40059464003209133, 0.4, 0.3120974540123476)
0.28141094098540903
Validation Loss Improved (0 ---> 0.28141094098540903)
Model Saved



[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|███████████████████████████████████████████| 375/375 [00:11<00:00, 31.38it/s, Epoch=2, LR=2.05e-7, Train_Loss=1.09]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|█████████████████████████████████████████████| 63/63 [00:00<00:00, 83.37it/s, Epoch=2, LR=2.05e-7, Valid_Loss=1.06]


...

Accuracy: 0.632
Precision: 0.648
Recall: 0.632
F_score: 0.637

              precision    recall  f1-score   support

           0       0.69      0.65      0.67       190
           1       0.45      0.57      0.50       113
           2       0.72      0.65      0.68       197

    accuracy                           0.63       500
   macro avg       0.62      0.62      0.62       500
weighted avg       0.65      0.63      0.64       500

(0.632, 0.6477129193071696, 0.632, 0.6373636205962059)
0.6182511291779584



[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|██████████████████████████████████████████| 375/375 [00:12<00:00, 30.57it/s, Epoch=99, LR=3.43e-7, Train_Loss=0.13]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|████████████████████████████████████████████| 63/63 [00:00<00:00, 78.98it/s, Epoch=99, LR=3.43e-7, Valid_Loss=1.16]


Accuracy: 0.636
Precision: 0.649
Recall: 0.636
F_score: 0.641

              precision    recall  f1-score   support

           0       0.68      0.66      0.67       190
           1       0.46      0.57      0.51       113
           2       0.72      0.65      0.68       197

    accuracy                           0.64       500
   macro avg       0.62      0.63      0.62       500
weighted avg       0.65      0.64      0.64       500

(0.636, 0.6492014989334989, 0.636, 0.6405262711145063)
0.6220750926633279



[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|████████████████████████████████████████████| 375/375 [00:12<00:00, 30.72it/s, Epoch=100, LR=1e-8, Train_Loss=0.12]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|██████████████████████████████████████████████| 63/63 [00:00<00:00, 80.61it/s, Epoch=100, LR=1e-8, Valid_Loss=1.16]


Accuracy: 0.636
Precision: 0.652
Recall: 0.636
F_score: 0.642

              precision    recall  f1-score   support

           0       0.69      0.65      0.67       190
           1       0.45      0.58      0.51       113
           2       0.73      0.65      0.69       197

    accuracy                           0.64       500
   macro avg       0.62      0.63      0.62       500
weighted avg       0.65      0.64      0.64       500

(0.636, 0.6524066547254574, 0.636, 0.6415088115212495)
0.6225876229784029

Training complete in 0h 22m 31s
Best Loss: 0.6280



In [41]:
class TestDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['#2_content'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        # summary = self.summary[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
                            
        
        text_ids = inputs_text['input_ids']
        text_mask = inputs_text['attention_mask']
        token_type_ids=inputs_text['token_type_ids']
        
       
        
        
        return {
            
            'input_ids': torch.tensor(text_ids, dtype=torch.long),
            'attention_mask': torch.tensor(text_mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }

In [42]:
from sklearn.metrics import jaccard_score,f1_score,accuracy_score,recall_score,precision_score,classification_report
def print_statistics(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    precision =precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')
    f_score = f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(classification_report(y, y_pred))
    return accuracy, precision, recall, f_score

In [43]:
# predict the test value result
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for data in tk0:
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        with torch.no_grad():
            y_preds = model(ids, mask)
        y_preds = (F.softmax(y_preds).to('cpu').numpy())
        # y_preds = y_preds.to('cpu').numpy()
        preds.append(y_preds)
    preds = np.concatenate(preds)
    return preds

In [44]:
model_pth=['/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Marbert/Loss_F1-Fold-0.bin','/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Marbert/Loss_F1-Fold-1.bin','/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Marbert/Loss_F1-Fold-2.bin','/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Marbert/Loss_F1-Fold-3.bin','/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Marbert/Loss_F1-Fold-4.bin']
tokenizer_marbert= AutoTokenizer.from_pretrained('UBC-NLP/MARBERTv2')

model_marbert =  NADIModel('UBC-NLP/MARBERTv2')

Some weights of the model checkpoint at UBC-NLP/MARBERTv2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
device="cuda:0"
testDataset = TestDataset(valid, tokenizer_marbert,max_length=256)
test_loader = DataLoader(testDataset,
                              batch_size =16,
                              shuffle=False,
                              num_workers = 4,
                              pin_memory = True,
                              drop_last=False)

marbert_predictions=[]
for pth in model_pth:
    model_marbert.load_state_dict(torch.load(pth))
    prediction = inference_fn(test_loader, model_marbert, device)
    marbert_predictions.append(prediction)
    torch.cuda.empty_cache()
    gc.collect()


marbert_predictions=np.argmax(np.mean(np.array(marbert_predictions),axis=0),axis=1)

  0%|                                                                                            | 0/32 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
  0%|                                                                                            | 0/32 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (functi

In [50]:
print(print_statistics(valid['#3_label'],marbert_predictions))


Accuracy: 0.656
Precision: 0.665
Recall: 0.656
F_score: 0.660

              precision    recall  f1-score   support

           0       0.71      0.69      0.70       190
           1       0.46      0.52      0.49       113
           2       0.74      0.70      0.72       197

    accuracy                           0.66       500
   macro avg       0.64      0.64      0.64       500
weighted avg       0.66      0.66      0.66       500

(0.656, 0.6647037772956753, 0.656, 0.6595877988045659)
